In [3]:
from PIL import Image
import torch
from torchvision.models import resnet18
from torchvision import transforms as T

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# input (single)
image = Image.open('cat.jpg')
transform = T.Compose([T.Resize((224, 224)), T.ToTensor()])
X = transform(image).unsqueeze(dim=0).to(device)

# original model
model = resnet18(weights="ResNet18_Weights.DEFAULT")
model = model.to(device)

# forward pass -- getting the outputs
out = model(X)

In [13]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
print(model.layer3[0].downsample[1])

BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [20]:
import torch.nn as nn
new_model = nn.Sequential(*list(model.children())[:-1])

In [29]:
new_model = nn.Sequential(*list(model.children())[:4])

In [32]:
new_model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [34]:
from PIL import Image
import torch
from torchvision.models import resnet18
from torchvision import transforms as T

# input (single)
image = Image.open('cat.jpg')
transform = T.Compose([T.Resize((224, 224)), T.ToTensor()])
X = transform(image).unsqueeze(dim=0).to(device)

# original model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

# a dict to store the activations
activation = {}
def getActivation(name):
  # the hook signature
  def hook(model, input, output):
    activation[name] = output.detach()
  return hook

# register forward hooks on the layers of choice
h1 = model.avgpool.register_forward_hook(getActivation('avgpool'))
h2 = model.maxpool.register_forward_hook(getActivation('maxpool'))
h3 = model.layer3[0].downsample[1].register_forward_hook(getActivation('comp'))

# forward pass -- getting the outputs
out = model(X)

print(activation)

# detach the hooks
h1.remove()
h2.remove()
h3.remove()

{'maxpool': tensor([[[[0.1451, 0.1460, 0.1460,  ..., 0.1475, 0.1475, 0.1475],
          [0.1499, 0.1591, 0.1682,  ..., 0.1546, 0.1546, 0.1546],
          [0.1499, 0.1591, 0.1682,  ..., 0.1546, 0.1546, 0.1546],
          ...,
          [0.2427, 0.4214, 0.4214,  ..., 0.3375, 0.4793, 0.3405],
          [0.2206, 0.4121, 0.4121,  ..., 0.3056, 0.3207, 0.3405],
          [0.1951, 0.4330, 0.5151,  ..., 0.3899, 0.5934, 0.6613]],

         [[0.3888, 0.3856, 0.3856,  ..., 0.3858, 0.3858, 0.3858],
          [0.3888, 0.3856, 0.3856,  ..., 0.3858, 0.3858, 0.3858],
          [0.3876, 0.3872, 0.3872,  ..., 0.3827, 0.3827, 0.3827],
          ...,
          [0.5727, 0.5321, 0.4999,  ..., 0.5577, 0.6612, 0.6460],
          [0.5321, 0.5321, 0.5449,  ..., 0.4146, 0.4931, 0.4931],
          [0.3881, 0.4452, 0.5449,  ..., 0.4404, 0.4580, 0.3922]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0

In [36]:
model.avgpool.register_forward_hook(getActivation('avgpool'))